### Kaggle Team Name: Team_04
##### MAD: 0.373
#### The 3rd Place on Kaggle


In [45]:
import pandas as pd
import numpy as np

## IMPORT DATA AND DELETE USELESS VARIABLES

In [46]:
df1 = pd.read_csv('raw-data_recipe.csv')

In [47]:
df1.head()

,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."


In [48]:
df1.drop(columns = 'image_url', axis = 1, inplace = True) # We don't need image_url for our models.

In [49]:
df1.head()

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."


In [50]:
df2 = pd.read_csv('core-data-train_rating.csv')
df2.drop(['dateLastModified'],axis = 1, inplace = True)

In [51]:
df2.head()

,user_id,recipe_id,rating
0,5215572,17991,5
1,5215572,170724,4
2,5215572,18045,5
3,3622615,60598,4
4,1313770,47519,5


## DATA CLEANING & MANIPULATIONS
### Recipe
    * ingredients
    * cooking_directions (prep, cooking, ready in time, if applicable)
    * nutritions (calories, calories from fat, carbohydrates, proteins, if applicable)
#### ingredients
The information in ingredients are not entirely 'ingredient', it could also involve some cooking instructions. For example:

In [52]:
df1.ingredients[99]
# Cox apple cored and cut into thick rings
# red apple cored and cut into rings

'butter^boneless pork loin chops^chopped fresh thyme^large onion^garlic^Cox apple cored and cut into thick rings^red apple cored and cut into rings^white sugar^salt and pepper to taste^dry hard cider^heavy cream'

In order to better serve our purpose, we decided to split the name of "ingredients" if they contain more than 2 words.

In [53]:
df1['Ingredients'] = [i.split('^') for i in df1.ingredients]

In [54]:
for i in df1.Ingredients:
    for j in i:
        if j.count(' ') > 2:
            i += j.split(' ')
            i.remove(j)

### cooking_directions
#### Get the Prep, Cook, and Ready In Time from Each Recipes
There are two types of cooking_directions, as followings:

In [55]:
df1.cooking_directions[99] # The one with the timing.

"{'directions': u'Prep\\n25 m\\nCook\\n40 m\\nReady In\\n1 h 5 m\\nPreheat oven to 375 degrees F (190 degrees C).\\nIn a large heavy skillet, over medium-high heat, melt half of the butter. Fry the pork chops on both sides until a nice golden color. Transfer the chops to a 2 quart casserole dish with a lid, and sprinkle with thyme.\\nMelt the remaining butter in the pan, and fry the onion and garlic until tender. Transfer them to the casserole dish. Fry the apple rings in the pan for a few seconds on each side to give them color, then remove to the casserole dish. Sprinkle with sugar.\\nSpoon any excess fat from the frying pan and pour in the apple cider. Stir to remove any flavorful bits from the bottom. Bring the mixture to a simmer, then pour over the pork chops and apples in the dish. Season with salt and pepper.\\nBake the casserole, covered for 30 to 40 minutes, or until the pork chops are cooked through. Remove the pork chops and apples to a serving platter and stir the cream in

In [56]:
df1.cooking_directions[58] # The one without the timing.

"{'directions': u'Preheat oven to 350 degrees F (175 degrees C).\\nHeat oil in a medium skillet over medium high heat. Add chicken breasts and saute until tender. Meanwhile, prepare potatoes according to package directions, adding 1 cup (instead of 2/3 cup) of milk and sliced potatoes. When chicken is tender, remove from heat, let cool and shred. Add shredded chicken to potato mixture.\\nFold chicken/potato mixture into a 9x13 inch baking dish. Cover and bake in the preheated oven for 25 to 35 minutes. Add cheese and bake for another 10 minutes.'}"

#### Also, the lengths of time in the instructions are in hours and minutes. For easier comparison, we need to convert them to the same format (in minute).

In [57]:
import re

In [58]:
def getmins(i):
    try:
        hour_to_min = int(str(re.search(r'\d+ h', i)).split("'")[1].split(" ")[0]) * 60
        mins = int(str(re.search(r'\d+ m', i)).split("'")[1].split(" ")[0])
        mins = hour_to_min + mins
    except IndexError:
        mins = int(str(re.search(r'\d+ m', i)).split("'")[1].split(" ")[0])
    return mins

In [59]:
lis_prep = []
for i in df1.cooking_directions:
    try:
        prep = str(i.split('Prep\\n')[1]).split('\\n')[0]
        lis_prep.append(getmins(prep))
    except IndexError:
        lis_prep.append(-1) # If there is no Prep time in directions, the time in dataframe will show as -1. 

In [60]:
df1['Prep'] = lis_prep

In [61]:
lis_cook = []
for i in df1.cooking_directions:
    try:
        cook = str(i.split('Cook\\n')[1]).split('\\n')[0]
        lis_cook.append(getmins(cook))
    except IndexError:
        lis_cook.append(-1) # If there is no Cook time in directions, the time in dataframe will show as -1.

In [62]:
df1['Cook'] = lis_cook

In [63]:
lis_ready = []
for i in df1.cooking_directions:
    try:
        ready = str(i.split('Cook\\n')[1]).split('\\n')[0]
        lis_ready.append(getmins(ready))
    except IndexError:
        lis_ready.append(-1) # If there is no Ready In in directions, the time in dataframe will show as -1.

In [64]:
df1['Ready'] = lis_ready 

#### In case prep/cook/ready time is 0, we used -1 for thoes don't have time infomation at all

#### nutritions
Currently, we only concern about the amount of calories, calories from fat, carbohydrates and proteins in each meal.

In [65]:
lis_cal = []
for i in df1.nutritions:
    try:
        cal = int(float(str(i.split("u'Calories', u'amount': ")[1]).split(", u'percentDailyValue'")[0]))
        lis_cal.append(cal)
    except IndexError:
        lis_cal.append(-1)

In [66]:
df1['Calories'] = lis_cal

In [67]:
lis_cal_fat = []
for i in df1.nutritions:
    try:
        cal = int(float(str(i.split("u'Calories from Fat', u'amount': ")[1]).split(", u'percentDailyValue'")[0]))
        lis_cal_fat.append(cal)
    except IndexError:
        lis_cal_fat.append(-1)

In [68]:
df1['Calories_Fat'] = lis_cal_fat

In [69]:
lis_carb = []
for i in df1.nutritions:
    try:
        carb = int(float(str(i.split("u'Carbohydrates', u'amount': ")[1]).split(", u'percentDailyValue'")[0]))
        lis_carb.append(carb)
    except IndexError:
        lis_carb.append(-1)

In [70]:
df1['Carbohydrates'] = lis_carb

In [71]:
lis_protein = []
for i in df1.nutritions:
    try:
        protein = int(float(str(i.split("u'Protein', u'amount': ")[1]).split(", u'percentDailyValue'")[0]))
        lis_protein.append(protein)
    except IndexError:
        lis_protein.append(-1)

In [72]:
df1['Protein'] = lis_protein

In [73]:
df1.drop(columns = ['ingredients', 'nutritions', 'reviews'], axis = 1, inplace = True) 
# We don't need these for our models, since we already got what we need.

In [74]:
df1.head()

,recipe_id,recipe_name,aver_rate,review_nums,cooking_directions,Ingredients,Prep,Cook,Ready,Calories,Calories_Fat,Carbohydrates,Protein
0,222388,Homemade Bacon,5.000000,3,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"[pork belly, smoked paprika, kosher salt, grou...",5,165,165,308,212,1,21
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"[sauerkraut drained, large onion, caraway seed...",15,150,150,371,105,32,36
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","[chicken wings, sprigs rosemary, head garlic, ...",20,40,40,335,212,6,23
3,87211,Chicken Pesto Paninis,4.625000,163,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"[focaccia bread quartered, prepared basil pest...",15,5,5,640,264,60,32
4,245714,Potato Bacon Pizza,4.500000,2,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"[red potatoes, strips bacon, Sauce:, heavy whi...",20,45,45,162,68,16,7


In [75]:
# combine user-rating records and recipe information together
df = pd.merge(df1, df2, how = 'right', on = 'recipe_id')

In [76]:
df.head()

,recipe_id,recipe_name,aver_rate,review_nums,cooking_directions,Ingredients,Prep,Cook,Ready,Calories,Calories_Fat,Carbohydrates,Protein,user_id,rating
0,218939,Foolproof Rosemary Chicken Wings,4.571429,12,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","[chicken wings, sprigs rosemary, head garlic, ...",20,40,40,335,212,6,23,2783111,5
1,87211,Chicken Pesto Paninis,4.625000,163,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"[focaccia bread quartered, prepared basil pest...",15,5,5,640,264,60,32,742713,4
2,87211,Chicken Pesto Paninis,4.625000,163,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"[focaccia bread quartered, prepared basil pest...",15,5,5,640,264,60,32,674114,5
3,87211,Chicken Pesto Paninis,4.625000,163,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"[focaccia bread quartered, prepared basil pest...",15,5,5,640,264,60,32,191706,4
4,87211,Chicken Pesto Paninis,4.625000,163,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"[focaccia bread quartered, prepared basil pest...",15,5,5,640,264,60,32,2193866,5


## FEATURE SELECTION

### Given a user_id: 
find all information of recipes this customer has rated, and then extract features based on these information and vectorize them

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

#### Numerical features:
    * Review_nums
    * Aver_rating
    * Time (Prep, Cook, Ready)
    * Nutritions (Calories, Calories_fat, Carbohydrates, Protein)

#### Text features: 
    * Recipe_name
    * Ingredients
    * Cooking_directions


In [78]:
# time-related features
time_features = lambda x: x[['Prep', 'Cook', 'Ready']].apply(lambda x: dict(x), axis=1)

# nutrition features
nutrition_features = lambda x: x[['Prep', 'Cook', 'Ready']].apply(lambda x: dict(x), axis=1)

# average rating and review number features
rating_features = lambda x: x[['aver_rate', 'review_nums']].apply(lambda x: dict(x), axis=1)

# ingredient features
ingredient_features = lambda x: x['Ingredients']

# recipe name feature
name_features = lambda x: x['recipe_name']

# cooking direction feature
directions_features = lambda x: x['cooking_directions']

In [79]:
# vectorize all features
time = make_pipeline(FunctionTransformer(time_features,validate=False),DictVectorizer())
nutrition = make_pipeline(FunctionTransformer(nutrition_features,validate=False),DictVectorizer())
avg_rating = make_pipeline(FunctionTransformer(rating_features,validate=False),DictVectorizer())
ingredient = make_pipeline(FunctionTransformer(ingredient_features,validate=False),TfidfVectorizer(analyzer='word',
                                                                                                   tokenizer=lambda x: x,
                                                                                                   preprocessor=lambda x: x,
                                                                                                   token_pattern=None))
name = make_pipeline(FunctionTransformer(name_features,validate=False),TfidfVectorizer(analyzer='word',
                                                                                       tokenizer=lambda x: x,
                                                                                       preprocessor=lambda x: x,
                                                                                       token_pattern=None))
direction = make_pipeline(FunctionTransformer(directions_features,validate=False),TfidfVectorizer(analyzer='word',
                                                                                                  tokenizer=lambda x: x,
                                                                                                  preprocessor=lambda x: x,
                                                                                                  token_pattern=None))

# combine all features
features = FeatureUnion([("time", time),
                         ("nutri",nutrition),
                         ("rating",avg_rating),
                         ("ingre",ingredient),
                         ("name", name),
                         ("dire",direction)])

## BUILD MODELS FOR EACH USER
After vectorizing recipe information of each user, we use KNN to build a model for each user. We also use GridSearch to test different parameters and get the optimal combination for our model with the highest prediction accuracy.

In [192]:
# i = user_id, j = recipe_id. This function is used to: 
# a. Given a user_id, find all information of recipes that the user has rated in the dataset
# b. Use "features"(the function created) to vectorize the information
# c. Try different parameters of KNN to find the best combination of parameters and get the optimized model
# d. Use the model to predict rating of the given recipe

def geprediction(i,j):
    user_df = df.loc[df['user_id'] == i, ['recipe_name', 'aver_rate', 'review_nums', 'rating', 'Ingredients', 'Prep', 'Cook', 'Ready',
                                          'Calories', 'Calories_Fat', 'Carbohydrates', 'Protein', 'cooking_directions']]
    y = user_df['rating']
    vecs = features.fit_transform(user_df)
    try:
        grid_param = {'n_neighbors': [20, 10, 5],      # when the user has rated more than 20 recipes
              'weights': ['distance', 'uniform'],
              'algorithm': ['brute', 'auto', 'ball_tree'],
              'leaf_size':[10, 30, 50]}
        gd_sr = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = grid_param, scoring = 'accuracy', cv = 5, n_jobs = -1)
        user_model = gd_sr.fit(vecs, y)
    except ValueError:
        grid_param = {'n_neighbors': [10, 5],          # when the user has rated more than 10 recipes
                      'weights': [ 'distance', 'uniform'],
                      'algorithm': ['brute', 'auto', 'ball_tree'],
                      'leaf_size':[10, 30, 50]}
        try:
            gd_sr = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = grid_param, scoring = 'accuracy', cv = 5, n_jobs = -1)
            user_model = gd_sr.fit(vecs, y)
        except ValueError:
            grid_param = {'n_neighbors': [5, 1],       # when the user has rated more than 5 recipes 
                          'weights': [ 'distance', 'uniform'],
                          'algorithm': ['brute', 'auto', 'ball_tree'],
                          'leaf_size':[10, 30, 50]}
            try:
                gd_sr = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = grid_param, scoring = 'accuracy', cv = 5, n_jobs = -1)
                user_model = gd_sr.fit(vecs, y)
            except ValueError:
                user_prediction = sum(y)/len(y)        # when the user has rated less than 5 recipes, use average rating as the prediction
                return user_prediction
    recipe_df = df1.loc[df1['recipe_id'] == j, ['recipe_name', 'aver_rate', 'review_nums', 'Ingredients', 'Prep', 'Cook', 'Ready',
                                                'Calories', 'Calories_Fat', 'Carbohydrates', 'Protein', 'cooking_directions']]
    recipe_vecs = features.transform(recipe_df)
    user_prediction = float(user_model.predict(recipe_vecs))
    return user_prediction

## GET PREDICTIONS

In [187]:
test = pd.read_csv('test_10k_1.csv')

In [188]:
test.head()

,user_id,recipe_id
0,5568,21412
1,3964202,75639
2,2621648,18015
3,3984678,12405
4,1055463,12051


In [189]:
lis_user = list(test['user_id'])
lis_recipe = list(test['recipe_id'])

In [195]:
import warnings
warnings.filterwarnings("ignore")   # remove warning messages 

##### We ran the following code in a different computer as it took too much time runing here 

In [ ]:
lis_prediction = [] 
for i,j in zip(lis_user, lis_recipe):
    lis_prediction.append(geprediction(i,j))

In [ ]:
test['rating'] = lis_prediction 

In [205]:
test['user_id'] = [str(i) for i in test.user_id]
test['recipe_id'] = [str(i) for i in test.recipe_id]
test['user_rep_id'] = test['user_id'] + test['recipe_id']

In [208]:
test[['user_rep_id', 'rating']].to_csv('result.csv', index = False, header = True)

### Thanks for reading!!!
Presented by Yiting Fu, Siying Liu & Liyuan Li